## Mounting Drive to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%mkdir /content/drive/MyDrive/last
%cd /content/drive/MyDrive/last/

/content/drive/MyDrive/last


## Cloning YOLOv5 and installing required packages

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd /content/drive/MyDrive/last/yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16582, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 16582 (delta 30), reused 39 (delta 18), pack-reused 16522
Receiving objects: 100% (16582/16582), 15.11 MiB | 13.36 MiB/s, done.
Resolving deltas: 100% (11383/11383), done.
Updating files: 100% (148/148), done.
/content/drive/MyDrive/last/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.5/756.5 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl 

## Install Roboflow for datasets

In [ ]:
!pip install roboflow

  Using cached roboflow-1.1.28-py3-none-any.whl (74 kB)
  Using cached certifi-2023.7.22-py3-none-any.whl (158 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached opencv_python_headless-4.8.0.74-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (49.1 MB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: cycler
    Found exi

### Datasets for 6 classes

In [ ]:
%cd /content/drive/MyDrive/last/
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("dark-mqa4m").project("waste-segregation-3ykjs")
version = project.version(3)
dataset = version.download("yolov5")

/content/drive/MyDrive/last
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to waste-segregation-3 in yolov5pytorch:: 100%|██████████| 11646/11646 [01:34<00:00, 122.85it/s]


### Dataset for wet and dry

In [ ]:
%cd /content/drive/MyDrive/last/
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("group7-9w9se").project("dry-waste-and-wet-waste")
version = project.version(1)
dataset = version.download("yolov5")

/content/drive/MyDrive/last
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Dry-waste-and-Wet-waste--1 in yolov5pytorch:: 100%|██████████| 17960/17960 [02:31<00:00, 118.75it/s]


## Segregate dataset based on classes for wet and dry

In [ ]:
import os
import shutil

def sort_dataset(images_dir,labels_dir,c_0_img,c_0_lab,c_1_img,c_1_lab):

    os.makedirs(c_0_img, exist_ok=True)
    os.makedirs(c_0_lab, exist_ok=True)
    os.makedirs(c_1_img, exist_ok=True)
    os.makedirs(c_1_lab, exist_ok=True)

    for filename in os.listdir(images_dir):
        if filename.endswith(".jpg"):  # Assuming images are JPEG format
            image_path = os.path.join(images_dir, filename)
            label_path = os.path.join(labels_dir, filename[:-4] + ".txt")

            # Read the annotation file
            with open(label_path, 'r') as file:
                annotations = file.readlines()

            # Check the class in annotations and move the image and label accordingly
            for annotation in annotations:
                class_id = int(annotation.split()[0])
                if class_id == 0:
                    shutil.copy(image_path, os.path.join(c_0_img, filename))
                    shutil.copy(label_path, os.path.join(c_0_lab, filename[:-4] + ".txt"))
                elif class_id == 1:
                    # Change class id to 6
                    annotation = '6' + annotation[1:]
                    # Save modified annotation to a new file
                    new_label_path = os.path.join(c_1_lab, filename[:-4] + ".txt")
                    with open(new_label_path, 'a') as new_file:
                        new_file.write(annotation)
                    shutil.copy(image_path, os.path.join(c_1_img, filename))


sort_dataset(
    images_dir=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train/images",
    labels_dir=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train/labels",
    c_0_img=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/class_0/images",
    c_0_lab=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/class_0/labels",
    c_1_img=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/images",
    c_1_lab=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/labels"
)



sort_dataset(
    images_dir=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid/images",
    labels_dir=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid/labels",
    c_0_img=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/class_0/images",
    c_0_lab=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/class_0/labels",
    c_1_img=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid_1/images",
    c_1_lab=r"/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid_1/labels"
)



In [ ]:
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/images/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/labels/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid_1/images/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid_1/labels/
%ls -l | wc -l

/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/images
3718
/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/labels
3718
/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid_1/images
280
/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid_1/labels
280


## Merge the dataset to add additional wet waste class

In [ ]:
import os
import random
import shutil

# Define the directories containing images and labels
images_dir = "/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/images/"
labels_dir = "/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/labels/"

# Define the output directories for the reduced dataset
output_images_dir = "/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/final_train_1/images/"
output_labels_dir = "/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/final_train_1/labels/"

# Create the output directories if they don't exist
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Get list of image and label files
image_files = os.listdir(images_dir)
label_files = os.listdir(labels_dir)

# Shuffle the files to randomly select samples
random.shuffle(image_files)

# Select first 1000 samples
selected_files = image_files[:1000]

# Copy selected images and corresponding labels to the output directories
for filename in selected_files:
    image_path = os.path.join(images_dir, filename)
    label_filename = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(labels_dir, label_filename)

    output_image_path = os.path.join(output_images_dir, filename)
    output_label_path = os.path.join(output_labels_dir, label_filename)

    shutil.copy(image_path, output_image_path)
    shutil.copy(label_path, output_label_path)


In [ ]:
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/final_train_1/images/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/final_train_1/labels/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/images/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/labels/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid_1/images/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid_1/labels/
%ls -l | wc -l

In [ ]:
%cd /content/drive/MyDrive/last/waste-segregation-3/train/images/
%rm glass*
%rm spray_cans*
%cd /content/drive/MyDrive/last/waste-segregation-3/train/labels/
%rm glass*
%rm spray_cans*

In [ ]:
%cd /content/drive/MyDrive/last/waste-segregation-3/train/images/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/waste-segregation-3/train/labels/
%ls -l | wc -l

/content/drive/MyDrive/last/waste-segregation-3/train/images
2909
/content/drive/MyDrive/last/waste-segregation-3/train/labels
2909


In [ ]:
%cp /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/final_train_1/images/* /content/drive/MyDrive/last/waste-segregation-3/train/images/
%cp /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/final_train_1/labels/* /content/drive/MyDrive/last/waste-segregation-3/train/labels/

In [ ]:
%cd /content/drive/MyDrive/last/waste-segregation-3/train/images/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/waste-segregation-3/train/labels/
%ls -l | wc -l

In [ ]:
%cp /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid_1/images/* /content/drive/MyDrive/last/waste-segregation-3/valid/images/
%cp /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/valid_1/labels/* /content/drive/MyDrive/last/waste-segregation-3/valid/labels/

In [ ]:
%cd /content/drive/MyDrive/last/waste-segregation-3/valid/images/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/waste-segregation-3/valid/labels/
%ls -l | wc -l

## EDIT data.yaml file

In [ ]:
# names: ["Cardboard","Glass","Metal","Paper","Plastic","Thermocol","Wet Waste"]

# nc: 7

# train: /content/drive/MyDrive/last/waste-segregation-3/train/images
# val: /content/drive/MyDrive/last/waste-segregation-3/valid/images


In [ ]:
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/images/
%ls -l | wc -l
%cd /content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/labels/
%ls -l | wc -l

/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/images
3718
/content/drive/MyDrive/last/Dry-waste-and-Wet-waste--1/train_1/labels
3718


## Make sure cuda is available

In [ ]:
import torch
torch.cuda.is_available()

True

## Start training by passing required arguments

In [ ]:
%cd /content/drive/MyDrive/last/yolov5/
!python train.py --img 640 --batch 16 --epochs 30 --data /content/drive/MyDrive/last/waste-segregation-3/data.yaml --weights yolov5m.pt

/content/drive/MyDrive/last/yolov5
2024-05-09 16:25:10.726369: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 16:25:10.726442: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 16:25:10.847865: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=, data=/content/drive/MyDrive/last/waste-segregation-3/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, c

## Once the training is done download the model file

In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/last/yolov5/runs/train/exp/weights/best.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("done")

done
